In [81]:
import pandas as pd
from datetime import datetime

# Load the raw data from the CSV
file_path = 'ClassSchedule.csv'
df_raw = pd.read_csv(file_path, encoding='latin1')

# Initialize lists to store course information
course_codes = []
course_titles = []
sessions = []
credits = []
class_times = []
instructors = []
days_list = []
start_times = []
end_times = []
cross_listings = []

# Function to map abbreviations to days and create columns for each day of the week
def map_days(abbreviation):
    days_columns = {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0, 'Friday': 0, 'Saturday': 0, 'Sunday': 0}
    
    if 'M' in abbreviation:
        days_columns['Monday'] = 1
    if 'T' in abbreviation and 'Th' not in abbreviation:
        days_columns['Tuesday'] = 1
    if 'T' in abbreviation and 'TTh' in abbreviation:
        days_columns['Tuesday'] = 1
        days_columns['Thursday'] = 1
    if 'Th' in abbreviation:
        days_columns['Thursday'] = 1
    if 'W' in abbreviation:
        days_columns['Wednesday'] = 1
    if 'F' in abbreviation:
        days_columns['Friday'] = 1
    if 'S' in abbreviation and 'Su' not in abbreviation:
        days_columns['Saturday'] = 1
    if 'S' in abbreviation and 'Su' in abbreviation:
        days_columns['Saturday'] = 1
        days_columns['Sunday'] = 1
    if 'Su' in abbreviation:
        days_columns['Sunday'] = 1
    if 'FSSu' in abbreviation:
        days_columns['Friday'] = 1
        days_columns['Saturday'] = 1
        days_columns['Sunday'] = 1
    return days_columns

# Function to convert time to 24-hour format
def convert_to_24_hour(time_str):
    return datetime.strptime(time_str, "%I:%M%p").strftime("%H:%M")

# Function to parse time strings and convert to 24-hour format
def parse_time(time_str):
    return convert_to_24_hour(time_str.replace('a', 'AM').replace('p', 'PM'))

# Loop through the dataframe to extract and recompose the data
for index, row in df_raw.iterrows():
    course_code = row['Course Code']
    course_title = row['Course Title']
    session = row['Session']
    credit = row['Credit']
    day_abbreviation = row['Class Time'].split()[0]
    time_range = row['Class Time'].split()[1]
    instructor = row['Instructor'] if pd.notna(row['Instructor']) else 'TBA'
    cross_listing = row['Crosslisting']
    
    # Map day abbreviation to full day names and create columns for each day of the week
    days_columns = map_days(day_abbreviation)
    days_list.append(days_columns)
    
    # Split time range into start and end times
    if '-' in time_range:
        start_time_str, end_time_str = time_range.split('-')
        start_time = parse_time(start_time_str)
        end_time = parse_time(end_time_str)
    else:
        start_time = None
        end_time = None
    
    # Append to the respective lists
    course_codes.append(course_code)
    course_titles.append(course_title)
    sessions.append(session)
    credits.append(credit)
    class_times.append(time_range)
    start_times.append(start_time)
    end_times.append(end_time)
    instructors.append(instructor)
    cross_listings.append(cross_listing)
    
    # If there's a Cross Listing Code, duplicate the entry
    if pd.notna(cross_listing):
        cross_listing_new = cross_listing + ' ' + course_code[-5:]
        course_code_new = course_code.replace(' ' + course_code[-5:], '')
        course_codes.append(cross_listing_new)
        course_titles.append(course_title)
        sessions.append(session)
        credits.append(credit)
        class_times.append(time_range)
        start_times.append(start_time)
        end_times.append(end_time)
        instructors.append(instructor)
        cross_listings.append(course_code_new)
        days_list.append(days_columns)

# Create a DataFrame from the lists
days_df = pd.DataFrame(days_list)
df_cleaned = pd.DataFrame({
    'Course Code': course_codes,
    'Course Title': course_titles,
    'Session': sessions,
    'Credit': credits,
    'Class Time': class_times,
    'Start Time': start_times,
    'End Time': end_times,
    'Instructor': instructors,
    'Cross Listing Code': cross_listings
})

# Combine the days DataFrame with the main DataFrame
df_cleaned = pd.concat([df_cleaned, days_df], axis=1)

# Save the DataFrame to a CSV file
output_file_path = 'CleanedClassSchedule.csv'
df_cleaned.to_csv(output_file_path, index=False)

output_file_path

'CleanedClassSchedule.csv'